In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp processing

In [ ]:
#| export
import re
from typing import Any, Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from pandas.tseries.offsets import BaseOffset

from utilsforecast.compat import DataFrame, Series, pl, pl_DataFrame, pl_Series
from utilsforecast.validation import validate_format

In [ ]:
from fastcore.test import test_eq
from nbdev import show_doc

from utilsforecast.compat import POLARS_INSTALLED
from utilsforecast.data import generate_series

In [ ]:
#| exporti
def _polars_categorical_to_numerical(serie: pl_Series) -> pl_Series:
    if serie.dtype == pl.Categorical:
        serie = serie.to_physical()
    return serie

In [ ]:
#| export
def to_numpy(df: DataFrame) -> np.ndarray:
    if isinstance(df, pd.DataFrame):
        cat_cols = [c for c, dtype in df.dtypes.items() if isinstance(dtype, pd.CategoricalDtype)]
        if cat_cols:
            df = df.copy(deep=False)
            for col in cat_cols:
                df[col] = df[col].cat.codes
        df = df.to_numpy()
    else:
        try:
            expr = pl.all().map_batches(_polars_categorical_to_numerical)
        except AttributeError:
            expr = pl.all().map(_polars_categorical_to_numerical)
        df = df.select(expr).to_numpy(order='c')
    return df

In [ ]:
#| export
def counts_by_id(df: DataFrame, id_col: str) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        id_counts = df.groupby(id_col, observed=True).size()
        if not id_counts.index.is_monotonic_increasing:
            id_counts = id_counts.sort_index()
        id_counts = id_counts.reset_index()
        id_counts.columns = [id_col, 'counts']
    else:
        id_counts = df[id_col].value_counts().sort(id_col)
    return id_counts

In [ ]:
#| export
def maybe_compute_sort_indices(
    df: DataFrame, id_col: str, time_col: str
) -> Optional[np.ndarray]:
    """Compute indices that would sort dataframe
            
    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times and target values.

    Returns
    -------
    numpy array or None
        Array with indices to sort the dataframe or None if it's already sorted.
    """
    if isinstance(df, pd.DataFrame):
        idx = pd.MultiIndex.from_frame(df[[id_col, time_col]])
    else:
        # this was faster than trying to build the multi index from polars
        sort_idxs = df.select(pl.arg_sort_by([id_col, time_col]).alias('idx'))['idx']
        idx = pd.Index(sort_idxs.to_numpy())
    if idx.is_monotonic_increasing:
        return None
    if isinstance(df, pd.DataFrame):
        sort_idxs = idx.argsort()
    return sort_idxs

In [ ]:
#| export
def assign_columns(df: DataFrame, names: Union[str, List[str]], values: Union[np.ndarray, pd.Series, pl_Series]) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        df[names] = values
    else:
        is_scalar = isinstance(values, str) or not hasattr(values, '__len__')
        if is_scalar:
            assert isinstance(names, str)
            vals: Union[pl_DataFrame, pl_Series, pl.Expr] = pl.lit(values).alias(names)
        elif isinstance(values, pl_Series):
            assert isinstance(names, str)
            vals = values.alias(names)
        else:
            if isinstance(names, str):
                names = [names]
            vals = pl.from_numpy(values, schema=names)
        df = df.with_columns(vals)
    return df

In [ ]:
engines = ['pandas']
if POLARS_INSTALLED:
    engines.append('polars')

In [ ]:
for engine in engines:
    series = generate_series(2, engine=engine)
    x = np.random.rand(series.shape[0])    
    series = assign_columns(series, 'x', x)
    series = assign_columns(series, ['y', 'z'], np.vstack([x, x]).T)
    series = assign_columns(series, 'ones', 1)
    series = assign_columns(series, 'zeros', np.zeros(series.shape[0]))
    series = assign_columns(series, 'as', 'a')
    np.testing.assert_allclose(
        series[['x', 'y', 'z']],
        np.vstack([x, x, x]).T
    )
    np.testing.assert_equal(series['ones'], np.ones(series.shape[0]))
    np.testing.assert_equal(series['as'], np.full(series.shape[0], 'a'))

In [ ]:
#| export
def take_rows(df: Union[DataFrame, Series, np.ndarray], idxs: np.ndarray) -> DataFrame:
    if isinstance(df, (pd.DataFrame, pd.Series)):
        df = df.iloc[idxs]
    else:
        df = df[idxs]
    return df

In [ ]:
for engine in engines:
    series = generate_series(2, engine=engine)
    subset = take_rows(series, np.array([0, 2]))
    assert subset.shape[0] == 2

In [ ]:
#| export
def filter_with_mask(
    df: Union[Series, DataFrame, pd.Index, np.ndarray],
    mask: Union[np.ndarray, pd.Series, pl_Series]
) -> DataFrame:
    if isinstance(df, (pd.DataFrame, pd.Series, pd.Index, np.ndarray)):
        out = df[mask]
    else:
        out = df.filter(mask)  # type: ignore
    return out

In [ ]:
#| export
def is_nan(s: Series) -> Series:
    if isinstance(s, pd.Series):
        out = s.isna()
    else:
        out = s.is_nan()
    return out

In [ ]:
np.testing.assert_equal(
    is_nan(pd.Series([np.nan, 1.0, None])).to_numpy(),
    np.array([True, False, True]),
)
if POLARS_INSTALLED:
    np.testing.assert_equal(
        is_nan(pl.Series([np.nan, 1.0, None])).to_numpy(),
        np.array([True, False, None]),
    )

In [ ]:
#| export
def is_none(s: Series) -> Series:
    if isinstance(s, pd.Series):
        out = is_nan(s)
    else:
        out = s.is_null()
    return out

In [ ]:
np.testing.assert_equal(
    is_none(pd.Series([np.nan, 1.0, None])).to_numpy(),
    np.array([True, False, True]),
)
if POLARS_INSTALLED:
    np.testing.assert_equal(
        is_none(pl.Series([np.nan, 1.0, None])).to_numpy(),
        np.array([False, False, True]),
    )

In [ ]:
#| export
def is_nan_or_none(s: Series) -> Series:
    return is_nan(s) | is_none(s)

In [ ]:
np.testing.assert_equal(
    is_nan_or_none(pd.Series([np.nan, 1.0, None])).to_numpy(),
    np.array([True, False, True]),
)
if POLARS_INSTALLED:
    np.testing.assert_equal(
        is_nan_or_none(pl.Series([np.nan, 1.0, None])).to_numpy(),
        np.array([True, False, True]),
    )

In [ ]:
#| export
def match_if_categorical(s1: Union[Series, pd.Index], s2: Series) -> Tuple[Series, Series]:
    if isinstance(s1.dtype, pd.CategoricalDtype):
        if isinstance(s1, pd.Index):
            cat1 = s1.categories
        else:
            cat1 = s1.cat.categories
        if isinstance(s2.dtype, pd.CategoricalDtype):
            cat2 = s2.cat.categories
        else:
            cat2 = s2.unique().astype(cat1.dtype)
        missing = set(cat2) - set(cat1)
        if missing:
            # we assume the original is s1, so we extend its categories
            new_dtype = pd.CategoricalDtype(categories=cat1.tolist() + sorted(missing))
            s1 = s1.astype(new_dtype)
            s2 = s2.astype(new_dtype)
    elif isinstance(s1, pl_Series) and s1.dtype == pl.Categorical:
        with pl.StringCache():
            cat1 = s1.cat.get_categories()
            if s2.dtype == pl.Categorical:
                cat2 = s2.cat.get_categories()
            else:
                cat2 = s2.unique().sort().cast(cat1.dtype)
            # populate cache, keep original categories first
            pl.concat([cat1, cat2]).cast(pl.Categorical)
            s1 = s1.cast(pl.Utf8).cast(pl.Categorical)
            s2 = s2.cast(pl.Utf8).cast(pl.Categorical)
    return s1, s2

In [ ]:
#| export
def vertical_concat(dfs: List[DataFrame], match_categories: bool = True) -> DataFrame:
    if not dfs:
        raise ValueError("Can't concatenate empty list.")
    if isinstance(dfs[0], pd.DataFrame):
        cat_cols = [c for c, dtype in dfs[0].dtypes.items() if isinstance(dtype, pd.CategoricalDtype)]
        if match_categories and cat_cols:
            if len(dfs) > 2:
                raise NotImplementedError('Categorical replacement for more than two dataframes')
            assert len(dfs) == 2
            df1, df2 = dfs
            df1 = df1.copy(deep=False)
            df2 = df2.copy(deep=False)            
            for col in cat_cols:
                s1, s2 = match_if_categorical(df1[col], df2[col])
                df1[col] = s1
                df2[col] = s2
            dfs = [df1, df2]
        out = pd.concat(dfs).reset_index(drop=True)
    else:
        all_cols = dfs[0].columns
        cat_cols = [all_cols[i] for i, dtype in enumerate(dfs[0].dtypes) if dtype == pl.Categorical]
        if match_categories and cat_cols:
            if len(dfs) > 2:
                raise NotImplementedError('Categorical replacement for more than two dataframes')
            assert len(dfs) == 2
            df1, df2 = dfs
            for col in cat_cols:
                s1, s2 = match_if_categorical(df1[col], df2[col])
                df1 = df1.with_columns(s1)
                df2 = df2.with_columns(s2)
            dfs = [df1, df2]
        out = pl.concat(dfs)
    return out

In [ ]:
df1 = pd.DataFrame({'x': ['a', 'b', 'c']}, dtype='category')
df2 = pd.DataFrame({'x': ['f', 'b', 'a']}, dtype='category')
pd.testing.assert_series_equal(
    vertical_concat([df1,df2])['x'],
    pd.Series(['a', 'b', 'c', 'f', 'b', 'a'], name='x', dtype=pd.CategoricalDtype(categories=['a', 'b', 'c', 'f']))
)

In [ ]:
#| polars
df1 = pl.DataFrame({'x': ['a', 'b', 'c']}, schema={'x': pl.Categorical})
df2 = pl.DataFrame({'x': ['f', 'b', 'a']}, schema={'x': pl.Categorical})
out = vertical_concat([df1,df2])['x']
assert out.series_equal(pl.Series('x', ['a', 'b', 'c', 'f', 'b', 'a']))
assert out.to_physical().series_equal(pl.Series('x', [0, 1, 2, 3, 1, 0]))
assert out.cat.get_categories().series_equal(
    pl.Series('x', ['a', 'b', 'c', 'f'])
)

In [ ]:
for engine in engines:
    series = generate_series(2, engine=engine)
    doubled = vertical_concat([series, series])
    assert doubled.shape[0] == 2 * series.shape[0]

In [ ]:
#| export
def horizontal_concat(dfs: List[DataFrame]) -> DataFrame:
    if not dfs:
        raise ValueError("Can't concatenate empty list.")
    if isinstance(dfs[0], pd.DataFrame):
        out = pd.concat(dfs, axis=1)
    elif isinstance(dfs[0], pl_DataFrame):
        out = pl.concat(dfs, how='horizontal')
    else:
        raise ValueError(f'Got list of unexpected types: {type(dfs[0])}.')        
    return out

In [ ]:
for engine in engines:
    series = generate_series(2, engine=engine)
    renamer = {c: f'{c}_2' for c in series.columns}
    if engine == 'pandas':
        series2 = series.rename(columns=renamer)
    else:
        series2 = series.rename(renamer)
    doubled = horizontal_concat([series, series2])
    assert doubled.shape[1] == 2 * series.shape[1]

In [ ]:
#| export
def copy_if_pandas(df: DataFrame, deep: bool = False) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        df = df.copy(deep=deep)
    return df

In [ ]:
#| export
def join(
    df1: Union[DataFrame, Series],
    df2: Union[DataFrame, Series],
    on: Union[str, List[str]],
    how: str = 'inner'
) -> DataFrame:
    if isinstance(df1, (pd.Series, pl_Series)):
        df1 = df1.to_frame()
    if isinstance(df2, (pd.Series, pl_Series)):
        df2 = df2.to_frame()
    if isinstance(df1, pd.DataFrame):
        out = df1.merge(df2, on=on, how=how)
    else:
        out = df1.join(df2, on=on, how=how)  # type: ignore
    return out

In [ ]:
#| export
def drop_index_if_pandas(df: DataFrame) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        df = df.reset_index(drop=True)
    return df

In [ ]:
#| export
def rename(df: DataFrame, mapping: Dict[str, str]) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        df = df.rename(columns=mapping, copy=False)
    else:
        df = df.rename(mapping)
    return df

In [ ]:
#| export
def sort(df: DataFrame, by: Optional[Union[str, List[str]]] = None) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        out = df.sort_values(by).reset_index(drop=True)
    elif isinstance(df, (pd.Series, pd.Index)):
        out = df.sort_values()
        if isinstance(out, pd.Series):
            out = out.reset_index(drop=True)
    elif isinstance(df, pl_DataFrame):
        out = df.sort(by)
    else:
        out = df.sort()
    return out

In [ ]:
pd.testing.assert_frame_equal(
    sort(pd.DataFrame({'x': [3, 1, 2]}), 'x'),
    pd.DataFrame({'x': [1, 2, 3]})
)
pd.testing.assert_frame_equal(
    sort(pd.DataFrame({'x': [3, 1, 2]}), ['x']),
    pd.DataFrame({'x': [1, 2, 3]})
)
pd.testing.assert_series_equal(
    sort(pd.Series([3, 1, 2])),
    pd.Series([1, 2, 3])
)
pd.testing.assert_index_equal(
    sort(pd.Index([3, 1, 2])),
    pd.Index([1, 2, 3])
)

In [ ]:
#| polars
# TODO: replace with pl.testing.assert_frame_equal when it's released
pd.testing.assert_frame_equal(
    sort(pl.DataFrame({'x': [3, 1, 2]}), 'x').to_pandas(),
    pd.DataFrame({'x': [1, 2, 3]}),
)
pd.testing.assert_frame_equal(
    sort(pl.DataFrame({'x': [3, 1, 2]}), ['x']).to_pandas(),
    pd.DataFrame({'x': [1, 2, 3]}),
)
pd.testing.assert_series_equal(
    sort(pl.Series('x', [3, 1, 2])).to_pandas(),
    pd.Series([1, 2, 3], name='x')
)

In [ ]:
#| export
def offset_dates(
    dates: Union[pd.Index, pl_Series],
    freq: Union[int, str, BaseOffset],
    n: int,
):
    if isinstance(dates, (pd.DatetimeIndex, pd.Series, pd.Index)) and isinstance(freq, (int, BaseOffset)):
        out = dates + n * freq
    elif isinstance(dates, pl_Series) and isinstance(freq, int):
        out = dates + n * freq
    elif isinstance(dates, pl_Series) and isinstance(freq, str):
        freq_n, freq_offset = re.findall(r'(\d+)(\w+)', freq)[0]
        total_n = int(freq_n) * n
        out = dates.dt.offset_by(f'{total_n}{freq_offset}')
    else:
        raise ValueError(f"Can't process the following combination {(type(dates), type(freq))}")
    return out

In [ ]:
#| export
def group_by(df: Union[Series, DataFrame], by, maintain_order=False):
    if isinstance(df, (pd.Series, pd.DataFrame)):
        out = df.groupby(by, observed=True, sort=not maintain_order)
    else:
        if isinstance(df, pl_Series):
            df = df.to_frame()
        try:
            out = df.group_by(by, maintain_order=maintain_order)
        except AttributeError:
            out = df.groupby(by, maintain_order=maintain_order)
    return out

In [ ]:
#| export
def group_by_agg(df: DataFrame, by, aggs, maintain_order=False) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        out = group_by(df, by, maintain_order).agg(aggs).reset_index()
    else:
        out = group_by(df, by, maintain_order).agg(*[getattr(pl.col(c), agg)() for c, agg in aggs.items()])
    return out

In [ ]:
pd.testing.assert_frame_equal(
    group_by_agg(pd.DataFrame({'x': [1, 1, 2], 'y': [1, 1, 1]}), 'x', {'y': 'sum'}),
    pd.DataFrame({'x': [1, 2], 'y': [2, 1]})
)

In [ ]:
#| polars
pd.testing.assert_frame_equal(
    group_by_agg(pl.DataFrame({'x': [1, 1, 2], 'y': [1, 1, 1]}), 'x', {'y': 'sum'}, maintain_order=True).to_pandas(),
    pd.DataFrame({'x': [1, 2], 'y': [2, 1]})
)

In [ ]:
#| export
def is_in(s: Series, collection) -> Series:
    if isinstance(s, pl_Series):
        out = s.is_in(collection)
    else:
        out = s.isin(collection)
    return out

In [ ]:
np.testing.assert_equal(is_in(pd.Series([1, 2, 3]), [1]), np.array([True, False, False]))

In [ ]:
#| polars
np.testing.assert_equal(is_in(pl.Series([1, 2, 3]), [1]), np.array([True, False, False]))

In [ ]:
#| export
def between(s: Series, lower: Series, upper: Series) -> Series:
    if isinstance(s, pd.Series):
        out = s.between(lower, upper)
    else:
        out = s.is_between(lower, upper)
    return out

In [ ]:
np.testing.assert_equal(
    between(pd.Series([1, 2, 3]), pd.Series([0, 1, 4]), pd.Series([4, 1, 2])),
    np.array([True, False, False]),
)

In [ ]:
#| polars
np.testing.assert_equal(
    between(pl.Series([1, 2, 3]), pl.Series([0, 1, 4]), pl.Series([4, 1, 2])),
    np.array([True, False, False]),
)

In [ ]:
#| export
def fill_null(df: DataFrame, mapping: Dict[str, Any]) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        out = df.fillna(mapping)
    else:
        out = df.with_columns(*[pl.col(col).fill_null(v) for col, v in mapping.items()])
    return out

In [ ]:
pd.testing.assert_frame_equal(
    fill_null(pd.DataFrame({'x': [1, np.nan], 'y': [np.nan, 2]}), {'x': 2, 'y': 1}),
    pd.DataFrame({'x': [1, 2], 'y': [1, 2]}, dtype='float64')
)

In [ ]:
#| polars
# TODO: replace with pl.testing.assert_frame_equal when it's released
pd.testing.assert_frame_equal(
    fill_null(pl.DataFrame({'x': [1, None], 'y': [None, 2]}), {'x': 2, 'y': 1}).to_pandas(),
    pd.DataFrame({'x': [1, 2], 'y': [1, 2]})
)

In [ ]:
#| export
def cast(s: Series, dtype: type) -> Series:
    if isinstance(s, pd.Series):
        s = s.astype(dtype)
    else:
        s = s.cast(dtype)
    return s

In [ ]:
pd.testing.assert_series_equal(
    cast(pd.Series([1, 2, 3]), 'int16'),
    pd.Series([1, 2, 3], dtype='int16')
)

In [ ]:
#| polars
pd.testing.assert_series_equal(
    cast(pl.Series('x', [1, 2, 3]), pl.Int16).to_pandas(),
    pd.Series([1, 2, 3], name='x', dtype='int16')
)

In [ ]:
#| export
def value_cols_to_numpy(
    df: DataFrame, id_col: str, time_col: str, target_col: str    
) -> np.ndarray:
    exclude_cols = [id_col, time_col, target_col]
    value_cols = [target_col] + [col for col in df.columns if col not in exclude_cols]
    data = to_numpy(df[value_cols])
    if data.dtype not in (np.float32, np.float64):
        data = data.astype(np.float32)
    return data

In [ ]:
#| export
def process_df(
    df: DataFrame,
    id_col: str,
    time_col: str,
    target_col: str
) -> Tuple[Series, np.ndarray, np.ndarray, np.ndarray, Optional[np.ndarray]]:
    """Extract components from dataframe
    
    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times and target values.

    Returns
    -------
    ids : pandas or polars Serie
        serie with the sorted unique ids present in the data.
    last_times : numpy array
        array with the last time for each serie.
    data : numpy ndarray
        2d array with target plus features values.
    indptr : numpy ndarray
        1d array with indices to the start and end of each serie.
    sort_idxs : numpy array or None
        array with the indices that would sort the original data.
        If the data is already sorted this is `None`.            
    """
    # validations
    validate_format(df, id_col, time_col, target_col)

    # ids
    id_counts = counts_by_id(df, id_col)
    uids = id_counts[id_col]

    # indices
    sizes = id_counts['counts'].to_numpy()
    indptr = np.append(0, sizes.cumsum()).astype(np.int32)
    last_idxs = indptr[1:] - 1

    # data
    data = value_cols_to_numpy(df, id_col, time_col, target_col)

    # check if we need to sort
    sort_idxs = maybe_compute_sort_indices(df, id_col, time_col)
    if sort_idxs is not None:
        data = data[sort_idxs]
        last_idxs = sort_idxs[last_idxs]
    times = df[time_col].to_numpy()[last_idxs]
    return uids, times, data, indptr, sort_idxs    

In [ ]:
#| export
class DataFrameProcessor:
    def __init__(
        self,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
    ):
        self.id_col = id_col
        self.time_col = time_col
        self.target_col = target_col

    def process(
        self,
        df: DataFrame
    ) -> Tuple[Series, np.ndarray, np.ndarray, np.ndarray, Optional[np.ndarray]]:
        return process_df(df, self.id_col, self.time_col, self.target_col)

In [ ]:
static_features = ['static_0', 'static_1']

In [ ]:
for n_static_features in [0, 2]:
    series_pd = generate_series(1_000, n_static_features=n_static_features, equal_ends=False, engine='pandas')
    for i in range(n_static_features):
        series_pd[f'static_{i}'] = series_pd[f'static_{i}'].map(lambda x: f'x_{x}').astype('category')
    scrambled_series_pd = series_pd.sample(frac=1.0)
    dfp = DataFrameProcessor('unique_id', 'ds', 'y')
    uids, times, data, indptr, _ = dfp.process(scrambled_series_pd)
    test_eq(times, series_pd.groupby('unique_id', observed=True)['ds'].max().values)
    test_eq(uids, np.sort(series_pd['unique_id'].unique()))
    for i in range(n_static_features):
        series_pd[f'static_{i}'] = series_pd[f'static_{i}'].cat.codes
    test_eq(data, series_pd[['y'] + static_features[:n_static_features]].to_numpy())
    test_eq(np.diff(indptr), series_pd.groupby('unique_id', observed=True).size().values)

In [ ]:
#| polars
for n_static_features in [0, 2]:
    series_pl = generate_series(1_000, n_static_features=n_static_features, equal_ends=False, engine='polars')
    scrambled_series_pl = series_pl.sample(fraction=1.0, shuffle=True)
    dfp = DataFrameProcessor('unique_id', 'ds', 'y')
    uids, times, data, indptr, _ = dfp.process(scrambled_series_pl)
    grouped = group_by(series_pl, 'unique_id')
    test_eq(times, grouped.agg(pl.col('ds').max()).sort('unique_id')['ds'].to_numpy())
    test_eq(uids, series_pl['unique_id'].unique().sort())
    test_eq(data, series_pl.select(pl.col(c).map_batches(lambda s: s.to_physical()) for c in ['y'] + static_features[:n_static_features]).to_numpy())
    test_eq(np.diff(indptr), grouped.count().sort('unique_id')['count'].to_numpy())